## Data processing

In [17]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    TrainerCallback,
)
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tqdm import tqdm
import torch.nn as nn

new_columns = [
    "par_id",      # 1 (integer ID)
    "art_id",      # @@24942188 (article identifier)
    "topic",       # hopeless (PCL category)
    "country",     # ph (country code)
    "text",        # Full text content
    "label"        # 0 (binary label)
]

# Read main dataset - skip 4 disclaimer rows
df = pd.read_csv(
    "data/dontpatronizeme_pcl.tsv",
    sep="\t",
    header=None,
    skiprows=4,
    names=new_columns,
    on_bad_lines='warn'
)

# Read train/dev splits
train_val_labels = pd.read_csv("data/train_semeval_parids-labels.csv")
test_labels = pd.read_csv("data/dev_semeval_parids-labels.csv")

# Convert string labels to lists
def parse_labels(label_str: str) -> list[int]:
    return [int(x) for x in label_str.strip("[]").replace(" ", "").split(",")]

# Process labels dataframes
for labels_df in [train_val_labels, test_labels]:
    labels_df['labels'] = labels_df['label'].apply(parse_labels)
    labels_df.drop('label', axis=1, inplace=True)

# Join with main data
train_val_df = df.merge(train_val_labels, on="par_id", how="inner")
test_df = df.merge(test_labels, on="par_id", how="inner")

# Add PCL positivity column to both dataframes
train_val_df['pcl_label'] = train_val_df['label'].apply(
    lambda x: 0 if x in {0, 1} else 1)
test_df['pcl_label'] = test_df['label'].apply(
    lambda x: 0 if x in {0, 1} else 1)

train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=42, shuffle=True)

### Hyperparameters

In [55]:
batch_size = 4
lr = 8e-5
n_epochs = 10
betas = (0.9, 0.98)
eps = 1e-6
wd = 1e-2

In [56]:
class PCLDataset(Dataset):
    def __init__(self, dataframe, tokenizer, balance_method='oversample'):
        self.tokenizer = tokenizer

         # Split into positive and negative classes
        pos_df = dataframe[dataframe['pcl_label'] == 1]
        neg_df = dataframe[dataframe['pcl_label'] == 0]

        # Balance classes
        if balance_method == 'oversample':
            # Repeat minority class samples
            if len(pos_df) > len(neg_df):
                pos_df, neg_df = neg_df, pos_df
            n_samples = max(len(pos_df), len(neg_df))
            pos_df = pos_df.sample(n_samples, replace=True, random_state=42)
        elif balance_method == 'undersample':
            # Take minimum number of samples
            n_samples = min(len(pos_df), len(neg_df))
            pos_df = pos_df.sample(n_samples, random_state=42)
            neg_df = neg_df.sample(n_samples, random_state=42)
        elif balance_method == 'None':
            pass

        # Combine and shuffle
        balanced_df = pd.concat([pos_df, neg_df]).sample(frac=1, random_state=42)
        self.texts = balanced_df['text'].tolist()
        self.labels = balanced_df['pcl_label'].tolist()

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]

        encoding = self.tokenizer(
            text,
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Initialize tokenizer and datasets
tokenizer = AutoTokenizer.from_pretrained("answerdotai/ModernBERT-base")

# Create datasets
train_dataset = PCLDataset(train_df, tokenizer)
val_dataset = PCLDataset(val_df, tokenizer, balance_method='None')
test_dataset = PCLDataset(test_df, tokenizer, balance_method='None')

## Weighted Random Sampler

In [57]:
class WeightedRandomSamplerTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.weights = torch.FloatTensor(self._get_weights())
        self.sampler = WeightedRandomSampler(self.weights, len(self.weights), replacement=True)

    def get_train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.args.train_batch_size, sampler=self.sampler, collate_fn=self.data_collator)

    def _get_weights(self):
        labels = np.array(self.train_dataset.labels)
        class_counts = np.bincount(labels)
        class_weights = 1.0 / np.sqrt(class_counts.astype(np.float32))
        weights = class_weights[labels]
        return weights

train_dataset = PCLDataset(train_df, tokenizer, 'None')
val_dataset = PCLDataset(val_df, tokenizer, balance_method='None')
test_dataset = PCLDataset(test_df, tokenizer, balance_method='None')

In [58]:
model_config = AutoConfig.from_pretrained("answerdotai/ModernBERT-base")
model_config.mlp_dropout = 0.2
model_config.num_labels = 2

# Initialize model with classification head
model = AutoModelForSequenceClassification.from_pretrained(
    "answerdotai/ModernBERT-base",
    num_labels=2,
)
model.train()
# Training setup
# device = torch.device("mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu")
# print("Using device:", device)
# model.to(device)
training_args = TrainingArguments(
    output_dir=f"ModernBERT_pcl_ft",
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=n_epochs,
    lr_scheduler_type="cosine",
    optim="adamw_torch_fused",
    adam_beta1=betas[0],
    adam_beta2=betas[1],
    adam_epsilon=eps,
    # weight_decay=wd,
    logging_strategy="epoch",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    bf16=True,
    bf16_full_eval=True,
    push_to_hub=False,
    warmup_ratio=0.1,

)

def compute_metrics(eval_pred):
    """Calculate classification metrics for Hugging Face Trainer"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, predictions, average='binary'
    )
    acc = accuracy_score(labels, predictions)

    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.training_history = {"train": [], "eval": []}

    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is not None:
            if "loss" in logs:  # Training logs
                self.training_history["train"].append(logs)
            elif "eval_loss" in logs:  # Evaluation logs
                self.training_history["eval"].append(logs)

trainer = WeightedRandomSamplerTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

metrics_callback = MetricsCallback()
trainer.add_callback(metrics_callback)

trainer.train()

train_history_df = pd.DataFrame(metrics_callback.training_history["train"])
train_history_df = train_history_df.add_prefix("train_")
eval_history_df = pd.DataFrame(metrics_callback.training_history["eval"])
train_res_df = pd.concat([train_history_df, eval_history_df], axis=1)

args_df = pd.DataFrame([training_args.to_dict()])

display(train_res_df)
display(args_df)

Some weights of ModernBertForSequenceClassification were not initialized from the model checkpoint at answerdotai/ModernBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.515700,0.525166,0.843582,0.478088,0.360360,0.710059
2,0.286100,0.523323,0.852537,0.488613,0.375796,0.698225
3,0.143700,0.559175,0.887761,0.525253,0.458150,0.615385
4,0.079000,0.472220,0.914030,0.414634,0.662338,0.301775
5,0.027700,1.094887,0.915224,0.360360,0.754717,0.236686
6,0.010100,1.215827,0.914627,0.375546,0.716667,0.254438
7,0.005800,1.291153,0.915224,0.366071,0.745455,0.242604
8,0.000000,1.451266,0.913433,0.319249,0.772727,0.201183
9,0.000000,1.446296,0.913433,0.331797,0.750000,0.213018
10,0.000000,1.450907,0.914627,0.341014,0.770833,0.218935


,train_loss,train_grad_norm,train_learning_rate,train_epoch,eval_loss,eval_accuracy,eval_f1,eval_precision,eval_recall
0,0.5157,3.290365e+00,0.000080,1.0,0.525166,0.843582,0.478088,0.360360,0.710059
1,0.2861,9.281063e-02,0.000078,2.0,0.523323,0.852537,0.488613,0.375796,0.698225
2,0.1437,9.733022e-03,0.000071,3.0,0.559175,0.887761,0.525253,0.458150,0.615385
3,0.0790,2.241047e-01,0.000060,4.0,0.472220,0.914030,0.414634,0.662338,0.301775
4,0.0277,1.510720e-04,0.000047,5.0,1.094887,0.915224,0.360360,0.754717,0.236686
5,0.0101,3.856534e-06,0.000033,6.0,1.215827,0.914627,0.375546,0.716667,0.254438
6,0.0058,1.145949e-06,0.000020,7.0,1.291153,0.915224,0.366071,0.745455,0.242604
7,0.0000,4.902281e-06,0.000009,8.0,1.451266,0.913433,0.319249,0.772727,0.201183
8,0.0000,5.168454e-06,0.000002,9.0,1.446296,0.913433,0.331797,0.750000,0.213018
9,0.0000,4.377248e-07,0.000000,10.0,1.450907,0.914627,0.341014,0.770833,0.218935


,output_dir,overwrite_output_dir,do_train,do_eval,do_predict,eval_strategy,prediction_loss_only,per_device_train_batch_size,per_device_eval_batch_size,per_gpu_train_batch_size,...,split_batches,include_tokens_per_second,include_num_input_tokens_seen,neftune_noise_alpha,optim_target_modules,batch_eval_metrics,eval_on_start,use_liger_kernel,eval_use_gather_object,average_tokens_across_devices
0,ModernBERT_pcl_ft,False,False,True,False,epoch,False,4,4,None,...,None,False,False,None,None,False,False,False,False,False


## Inference

In [53]:
import torch.nn.functional as F

checkpoint_path = "./ModernBERT_pcl_ft/checkpoint-838"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)


# Evaluation on a single example
def predict_single(text: str, model, tokenizer, device='cuda'):
    model.to(device)
    model.eval()

    encoding = tokenizer(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    return F.softmax(logits).cpu().numpy()

test_input = df[df['label'] == 3]['text'].iloc[3]
print(test_input)
predict_single(test_input, model, tokenizer)

The demographics of Pakistan and India are very similar . Poverty is a widespread issue . According to the FAO , 40 percent of children in Pakistan are malnourished and underweight due to lack of access to adequate food . And this is not because there is n't enough ; Pakistan is the 8th largest food producing country , however , 50 percent of the population is food insecure . With the massive income inequality that persists , RHA is a brilliant movement . We collect leftover or extra food from restaurants and distribute it to the homeless and hungry in the locality .


<ipython-input-53-6923b88e0452>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.softmax(logits).cpu().numpy()


array([[3.5850875e-04, 9.9964154e-01]], dtype=float32)

In [6]:
# from huggingface_hub import notebook_login

# notebook_login()

In [47]:
model_name = "Hasasasaki/modernBERT_pcl_ft"
model.push_to_hub(model_name)
tokenizer.push_to_hub(model_name)

model.safetensors:   0%|          | 0.00/598M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Hasasasaki/modernBERT_pcl_ft/commit/7480824b6f6a408e65e5c204578360ed06cc219e', commit_message='Upload tokenizer', commit_description='', oid='7480824b6f6a408e65e5c204578360ed06cc219e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Hasasasaki/modernBERT_pcl_ft', endpoint='https://huggingface.co', repo_type='model', repo_id='Hasasasaki/modernBERT_pcl_ft'), pr_revision=None, pr_num=None)

In [54]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    processing_class=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)
trainer.evaluate(test_dataset)

{'eval_loss': 0.3086358606815338,
 'eval_model_preparation_time': 0.0034,
 'eval_accuracy': 0.9235547061634019,
 'eval_f1': 0.5209580838323353,
 'eval_precision': 0.6444444444444445,
 'eval_recall': 0.4371859296482412,
 'eval_runtime': 5.2317,
 'eval_samples_per_second': 400.062,
 'eval_steps_per_second': 25.04}